In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
import numpy as np

## Load Pickles of checkpoint charges per byte

In [3]:
from os import path, pardir, scandir
import pickle
import sys
from math import ceil

In [4]:
from pydgilib_extra import LOGGER_CSV

In [5]:
sys.path.insert(0,'../..')

In [6]:
measure_security = False
show_lm_plot=2
measure_workloads = True
repetitions = 100

In [7]:
from experiments.checkpoint_energy import CheckpointEnergy

In [8]:
text_as_size = False
security_method = 'mbedTLS'
#checkpoint_energy = CheckpointEnergy()
# checkpoint_energy = CheckpointEnergy(security_projects=['AES_Flash', 'TrustZone_Flash', 'No_Security_Flash'])
checkpoint_energy = CheckpointEnergy(workload_folder = "../../Workloads",
                                     security_projects=['AES-256_mbedTLS_library_O1_4Mhz',
                                                        'AES-256_mbedTLS_library_O1_12Mhz',
                                                        'AES-256_mbedTLS_library_O3_4Mhz',
                                                        'AES-256_mbedTLS_library_O3_12Mhz'], text_as_size=text_as_size)

In [9]:
checkpoint_energy.security_projects

['AES-256_mbedTLS_library_O1_4Mhz',
 'AES-256_mbedTLS_library_O1_12Mhz',
 'AES-256_mbedTLS_library_O3_4Mhz',
 'AES-256_mbedTLS_library_O3_12Mhz']

In [10]:
checkpoint_energy.workload_projects

['BitCounting', 'Dijkstra', 'FFT', 'Sorting']

### Smoothen?

Get lookup function

In [11]:
get_security_energy = checkpoint_energy.get_security_energy_function()

In [12]:
for security_type in checkpoint_energy.security_projects + ["None"]:
    print(security_type, get_security_energy(security_type,2048))
    print(security_type, get_security_energy(security_type,2048, use_model=True))
    print(security_type, get_security_energy(security_type,100000000))

AES-256_mbedTLS_library_O1_4Mhz {'AES-256 Encrypt': 5.5073238537407735e-05, 'AES-256 Decrypt': 5.7610155677824296e-05, 'AES-256 Flash Write': 7.80541766042196e-05, 'AES-256 Flash Read': 2.0088334449804605e-06}
AES-256_mbedTLS_library_O1_4Mhz {'AES-256 Encrypt': 5.4699074098274594e-05, 'AES-256 Decrypt': 5.7051506116837106e-05, 'AES-256 Flash Write': 7.393299376565765e-05, 'AES-256 Flash Read': 1.9978475767044544e-06}
AES-256_mbedTLS_library_O1_4Mhz {'AES-256 Encrypt': 2.6865245970372698, 'AES-256 Decrypt': 2.7837382964179866, 'AES-256 Flash Write': 3.427248517127214, 'AES-256 Flash Read': 0.09985493607481974}
AES-256_mbedTLS_library_O1_12Mhz {'AES-256 Encrypt': 5.027556848108576e-05, 'AES-256 Decrypt': 5.291071056374187e-05, 'AES-256 Flash Write': 9.082222840657206e-05, 'AES-256 Flash Read': 1.5675312545966353e-06}
AES-256_mbedTLS_library_O1_12Mhz {'AES-256 Encrypt': 4.989618064626563e-05, 'AES-256 Decrypt': 5.2416269043851956e-05, 'AES-256 Flash Write': 8.455329234666236e-05, 'AES-256

In [58]:
import numpy as np
from math import sqrt, floor
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.ticker as tck


In [14]:
c_scale = 1e3
j_scale = 1e6
t_scale = 1e3
voltage = 3.31

In [15]:
checkpoint_energy.security_projects

['AES-256_mbedTLS_library_O1_4Mhz',
 'AES-256_mbedTLS_library_O1_12Mhz',
 'AES-256_mbedTLS_library_O3_4Mhz',
 'AES-256_mbedTLS_library_O3_12Mhz']

In [16]:
workload_colors = {'BitCounting':'tab:blue', 'Dijkstra':'tab:green', 'FFT':'tab:red', 'Sorting':'tab:orange'}
# security_colors = {'AES_Flash':'#85e085', 'TrustZone_Flash':'#66b3ff', 'No_Security_Flash':'#b3b3cc'}
security_colors = {'AES-256_mbedTLS_library_O1_4Mhz':'#258e25', 
                   'AES-256_mbedTLS_library_O1_12Mhz':'#258e25',
                   'AES-256_mbedTLS_library_O3_4Mhz':'#258e25',
                   'AES-256_mbedTLS_library_O3_12Mhz':'#258e25'}
colors = {**workload_colors, **security_colors}

In [17]:
hatches = ['-', '+', 'x', '\\', '*', 'o', 'O', '.']

In [18]:
section_hatches = {'AES Encrypt': '\\\\', 
                   'AES Decrypt': '//', 
                   'AES-256 Encrypt': '\\\\', 
                   'AES-256 Decrypt': '//', 
                   'AES-128 Encrypt': '\\\\', 
                   'AES-128 Decrypt': '//', 
                   'Flash Write': 'xx', 
                   'Flash Read': '/////', 
                   'AES-256 Flash Write': 'xx', 
                   'AES-256 Flash Read': '/////', 
                   'AES-128 Flash Write': 'xx', 
                   'AES-128 Flash Read': '/////', 
                   'No Security Flash Write': 'xx', 
                   'No Security Flash Read': '/////', 
                   'TrustZone Flash Write': 'xx', 
                   'TrustZone Flash Read': '/////', 
                   'Bit Counting': 'ooo', 
                   'Create Graph': '0', 
                   'Dijkstra': 'ooo',
                   'Allocate FFT': '0',
                   'FFT': 'ooo',
                   'Free FFT': '...',
                   'Sorting': 'ooo'
                  }


circ1 = mpatches.Patch( facecolor='None', hatch=r'\\\\',label='AES CBC Mode Encrypt')
circ2 = mpatches.Patch( facecolor='None', hatch='//',label='AES CBC Mode Decrypt')
circ3 = mpatches.Patch( facecolor='None', hatch='xx',label='Flash Write')
circ4 = mpatches.Patch( facecolor='None', hatch='/////',label='Flash Read')
        

In [77]:
edgecolor = 'black'
linewidth = 0.8

In [78]:
security_projects=['AES-256_mbedTLS_library_O1_4Mhz',
                                                        'AES-256_mbedTLS_library_O1_12Mhz',
                                                        'AES-256_mbedTLS_library_O3_4Mhz',
                                                        'AES-256_mbedTLS_library_O3_12Mhz']

In [79]:
xLabels = ['O1\n4MHz','O1\n12MHz','O3\n4MHz',
            'O3\n12MHz']

In [80]:
#security_projects=['AES-128_Flash', 'AES-256_Flash', 'TrustZone_Flash', 'No_Security_Flash']

In [81]:
totalEnergy = get_security_energy('AES-256_mbedTLS_library_O1_4Mhz',4096, use_model=True)
sum =0
for evl in totalEnergy.values():
    sum+=evl*j_scale*voltage
sum    

1231.1459416277053

In [82]:
class WorkloadPlotter(object):
    def __init__(self, checkpoint_energy=CheckpointEnergy(workload_folder = "../../Workloads",security_projects=security_projects), fig=None, ax=None, show=False):
        self.checkpoint_energy = checkpoint_energy
        self.fig = fig
        self.ax = ax
        if show:
            self.make_fig()
            self.update_fig()
            
        
    def make_fig(self, *args, **kwargs):
        self.fig, self.ax = plt.subplots(*args, **kwargs)
    
    def update_fig(self):
        opacity = 0.8
        self.ax.cla()
        
        self.bars = {}
        for index, workload_project in enumerate(self.checkpoint_energy.workload_projects):
            self.bars[workload_project] = {}
            section_averages = self.checkpoint_energy.get_workload_average(workload_project)
            section_stds = self.checkpoint_energy.get_workload_std(workload_project)
            bottom = 0
            for section in section_averages.keys():
                self.bars[workload_project][section] = self.ax.bar(index, section_averages[section]*j_scale*voltage, color=colors[workload_project], 
                                                                   label=section, alpha=opacity, yerr=section_stds[section]*j_scale*voltage, bottom=bottom, 
                                                                   hatch=section_hatches[section], linewidth=linewidth, edgecolor=edgecolor)
                bottom += section_averages[section]*j_scale*voltage
        
        self.ax.set_xticks(range(len(self.checkpoint_energy.workload_projects)))
        self.ax.set_xticklabels(self.checkpoint_energy.workload_projects)
        self.ax.legend()
#         self.ax.set_yscale('log')
        # self.ax.grid()

        self.ax.set_xlabel('Workload')
        self.ax.set_ylabel('Energy [$\mu$J]')
        self.ax.set_title('Energy per Section')
        self.fig.canvas.set_window_title('Energy per Workload')
        self.fig.show()

In [83]:
workload_plotter = WorkloadPlotter(checkpoint_energy, show=True)

In [84]:
# workload_plotter.make_fig()

In [85]:
# workload_plotter.update_fig()

In [86]:
checkpoint_energy.security_projects
#checkpoint_energy.get_security_energy('AES-256_Crypto-Accelerator', 4096)

['AES-256_mbedTLS_library_O1_4Mhz',
 'AES-256_mbedTLS_library_O1_12Mhz',
 'AES-256_mbedTLS_library_O3_4Mhz',
 'AES-256_mbedTLS_library_O3_12Mhz']

In [87]:
def set_size(w,h, ax=None):
    """ w, h: width, height in inches """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)


In [88]:
def set_xmargin(ax, left=0.0, right=0.3):
    ax.set_xmargin(0)
    ax.autoscale_view()
    lim = ax.get_xlim()
    delta = np.diff(lim)
    left = lim[0] - delta*left
    right = lim[1] + delta*right
    ax.set_xlim(left,right)

In [91]:
class SecurityPlotter(object):
    def __init__(self, checkpoint_energy=CheckpointEnergy(workload_folder = "../../Workloads",security_projects=security_projects), num_bytes=4096, fig=None, ax=None, show=0, use_model=False):
        self.checkpoint_energy = checkpoint_energy
        self.fig = fig
        self.ax = ax
        self.num_bytes = num_bytes
        self.use_model = use_model
        if show:
            self.make_fig(figsize=(12.8, 6.8),dpi= 300) #width, height in inches
            self.update_fig()
            if show >= 2:
                self.animate()
        
    def make_fig(self, *args, **kwargs):
        self.fig, self.ax = plt.subplots(*args, **kwargs)
    
    def update_fig(self, num_bytes=None):
        if num_bytes is None:
            num_bytes = self.num_bytes
        opacity = 0.8
        self.ax.cla()
#         self.ax.grid(zorder=0)
        self.ax.yaxis.grid(True, alpha=0.3)
        
        self.bars = {}
        for index, security_project in enumerate(self.checkpoint_energy.security_projects):
            self.bars[security_project] = {}
            security_energy = self.checkpoint_energy.get_security_energy(security_project, num_bytes, use_model=self.use_model)
            #print(f'{security_energy}')
            bottom = 0
            sectionEnergies = get_security_energy(security_project,num_bytes, use_model=self.use_model)
            totalEnergy = 0
            for sectionEnergy in sectionEnergies.values():
                totalEnergy+=sectionEnergy * j_scale * voltage
            #print(f'{totalEnergy}')
            for section in security_energy.keys():
                self.bars[security_project][section] = self.ax.bar(index, security_energy[section] * j_scale*voltage, color=colors[security_project], 
                                                                   label=section, width=0.4, alpha=opacity, bottom=bottom, zorder=3,
                                                                   hatch=section_hatches[section], linewidth=linewidth, edgecolor=edgecolor)
                sectionValue = security_energy[section] * j_scale*voltage
                accumulatedValue = sectionValue + bottom
                yValue = (totalEnergy - accumulatedValue/2)
                bar = self.bars[security_project][section].get_children()
                print(f'{index} : {sectionValue}')
                self.ax.text(x=index + (bar[0].get_width()*1.01 ) , y= bottom + sectionValue/2 -15 , 
                             s=f"{round(sectionValue)}", ha = 'center', va = 'center', fontdict=dict(fontsize=11))
                bottom += security_energy[section]*j_scale*voltage
                
            self.ax.text(x=index + 0.05 , y =1250+60 , s=f"{round(bottom, 2)}" ,bbox=dict(boxstyle="square, pad=0.03", fc="none", ec="gray", zorder=0.9, lw=0.5),
                         va='baseline', horizontalalignment='center', fontdict=dict(fontsize=12))
    
            print(f'{security_project}: {bottom}')        
        
        self.ax.set_xticks(range(len(self.checkpoint_energy.security_projects)) )
     ##   self.ax.set_xticklabels([s.replace('_',' ') for s in self.checkpoint_energy.security_projects])
        self.ax.set_xticklabels(xLabels, fontdict=dict(fontsize=12))
        self.ax.yaxis.set_tick_params(labelsize=12)

     #   self.ax.set_xticklabels([" ".join(s.split('_')[:-1]) for s in self.checkpoint_energy.security_projects])
       
      #  self.ax.set_yticks( fontdict=dict(fontsize=4))
      #  self.ax.legend(handles = [circ1,circ2,circ3, circ4],loc=6, prop={'size': 4})
       
      #  self.ax.set_yscale('log')

       # self.ax.set_xlabel('\nDifferent AES-Modes execution on AES-Module')
        self.ax.set_ylabel('Energy [$\mu$J]', fontdict=dict(fontsize=12))
        self.ax.set_title(f'(c) Security: mbedtLS (Software-based)' ,  fontdict=dict(fontsize=12))
        self.fig.canvas.set_window_title('Energy_per_parameter_mbedTLS')
        self.ax.margins( y= 0.12)
        self.ax.yaxis.set_minor_locator(tck.AutoMinorLocator())

       # self.ax.set(adjustable="datalim")
        #ratio = 0.5
        #xleft, xright = self.ax.get_xlim()
        #ybottom, ytop = self.ax.get_ylim()
        # the abs method is used to make sure that all numbers are positive
        # because x and y axis of an axes maybe inversed.
        #self.ax.set_aspect(abs((xright-xleft)/(ybottom-ytop))*ratio)
        
        set_xmargin(self.ax, left=0.05, right=0.11)
        #set_size(6.4,12.4, self.ax)
        plt.gcf().set_size_inches(5, 7)

        self.fig.tight_layout()
        
        self.fig.savefig(f'Energy_per_parameter_{security_method}.pdf', bbox_inches='tight', dpi=1000)
        self.fig.savefig(f'Energy_per_parameter_{security_method}.svg', bbox_inches='tight', dpi=1000)
        self.fig.show()
    def anim_func(self, num_bytes, *fargs):
        self.update_fig(num_bytes)
        return flatten(self.bars)
    
    def animate(self, frames=range(100, 6001, 100), save=False, *save_args, **save_kwargs):
        self.animation = animation.FuncAnimation(self.fig, self.anim_func, blit=True, interval=0,frames=frames ,repeat=False)
        if save:
            self.animation.save('Charge_per_Security.mp4', *save_args, **save_kwargs)
        
        
def flatten(d, ret=None):
    if ret is None:
        ret = []
    for k, v in sorted(d.items()):
        if isinstance(v, dict):
            flatten(v, ret)
        else:
            ret.extend(v.get_children())
    return ret

In [92]:
security_plotter = SecurityPlotter(checkpoint_energy, show=1, use_model=False)
# security_plotter = SecurityPlotter(checkpoint_energy, show=1, use_model=True)

0 : 363.936813054608
0 : 379.36258255206087
0 : 490.81148244310197
0 : 13.471643320021066
AES-256_mbedTLS_library_O1_4Mhz: 1247.582521369792
1 : 334.0481402452019
1 : 350.82517875037803
1 : 563.5293466142018
1 : 11.19956923343456
AES-256_mbedTLS_library_O1_12Mhz: 1259.6022348432164
2 : 364.67927726455684
2 : 380.08711604107214
2 : 489.00721266279197
2 : 13.444934042476618
AES-256_mbedTLS_library_O3_4Mhz: 1247.2185400108976
3 : 331.30023656011775
3 : 344.2710086835901
3 : 555.1188289300077
3 : 10.988472865797144
AES-256_mbedTLS_library_O3_12Mhz: 1241.6785470395125


## Security Time

In [93]:
def get_security_time(project_name, num_bytes):
    time_dict = {}
    project_path = [".", project_name]
    encrypt_time_ms = pickle.load(open(path.join(*project_path, "aes_flash_encrypt_time_ms.p"), "rb"))
    decrypt_time_ms = pickle.load(open(path.join(*project_path, "aes_flash_decrypt_time_ms.p"), "rb"))
    flash_write_time_ms = pickle.load(open(path.join(*project_path, "aes_flash_write_time_ms.p"), "rb"))
    flash_read_time_ms = pickle.load(open(path.join(*project_path, "aes_flash_read_time_ms.p"), "rb"))
    
    i = round((num_bytes/16) - 1)
    print(f'{i}')
    time_dict['AES-256 Encrypt'] = encrypt_time_ms[i]
    time_dict['AES-256 Decrypt'] = decrypt_time_ms[i]
    time_dict['AES-256 Flash Write'] =  flash_write_time_ms[i]
    time_dict['AES-256 Flash Read'] = flash_read_time_ms[i]
    
    return time_dict

In [94]:
class SecurityPlotter(object):
    def __init__(self, checkpoint_energy=CheckpointEnergy(workload_folder = "../../Workloads",security_projects=security_projects), num_bytes=4096, fig=None, ax=None, show=0, use_model=False):
        self.checkpoint_energy = checkpoint_energy
        self.fig = fig
        self.ax = ax
        self.num_bytes = num_bytes
        self.use_model = use_model
        if show:
            self.make_fig(figsize=(12.8, 6.8),dpi= 300) #width, height in inches
            self.update_fig()
            if show >= 2:
                self.animate()
        
    def make_fig(self, *args, **kwargs):
        self.fig, self.ax = plt.subplots(*args, **kwargs)
    
    def update_fig(self, num_bytes=None):
        if num_bytes is None:
            num_bytes = self.num_bytes
        opacity = 0.8
        self.ax.cla()
#         self.ax.grid(zorder=0)
        self.ax.yaxis.grid(True, alpha=0.3)
        
        self.bars = {}
        for index, security_project in enumerate(self.checkpoint_energy.security_projects):
            self.bars[security_project] = {}
            security_time = get_security_time(security_project, num_bytes)
            #print(f'{security_energy}')
            bottom = 0
            sectionTimes = get_security_time(security_project,num_bytes)
            totalTime = 0
            for sectionTime in sectionTimes.values():
                totalTime+=sectionTime
            #print(f'{totalEnergy}')
            for section in security_time.keys():
                self.bars[security_project][section] = self.ax.bar(index, security_time[section] , color=colors[security_project], 
                                                                   label=section, width=0.4, alpha=opacity, bottom=bottom, zorder=3,
                                                                   hatch=section_hatches[section], linewidth=linewidth, edgecolor=edgecolor)
                sectionValue = security_time[section] 
                accumulatedValue = sectionValue + bottom
                yValue = (totalTime - accumulatedValue/2)
                bar = self.bars[security_project][section].get_children()
                print(f'{index} : {sectionValue}')
                self.ax.text(x=index + (bar[0].get_width()*0.95 ) , y= bottom + sectionValue/2 -2 , 
                             s=f"{round(sectionValue)}", ha = 'center', va = 'center', fontdict=dict(fontsize=11))
                bottom += security_time[section]
                
            self.ax.text(x=index + 0.01 , y =bottom+20 , s=f"{round(bottom, 2)}" ,bbox=dict(boxstyle="square, pad=0.03", fc="none", ec="gray", zorder=0.9, lw=0.5),
                         va='baseline', horizontalalignment='center', fontdict=dict(fontsize=12))
    
            print(f'{security_project}: {bottom}')        
        
        self.ax.set_xticks(range(len(self.checkpoint_energy.security_projects)) )
     ##   self.ax.set_xticklabels([s.replace('_',' ') for s in self.checkpoint_energy.security_projects])
        self.ax.set_xticklabels(xLabels, fontdict=dict(fontsize=12))
        self.ax.yaxis.set_tick_params(labelsize=12)

     #   self.ax.set_xticklabels([" ".join(s.split('_')[:-1]) for s in self.checkpoint_energy.security_projects])
       
      #  self.ax.set_yticks( fontdict=dict(fontsize=4))
      #  self.ax.legend(handles = [circ1,circ2,circ3, circ4],loc=6, prop={'size': 4})
       
      #  self.ax.set_yscale('log')

       # self.ax.set_xlabel('\nDifferent AES-Modes execution on AES-Module')
        self.ax.set_ylabel('Time [ms]', fontdict=dict(fontsize=12))
        self.ax.set_title(f'(c) Security: {security_method} (Software-based)' ,  fontdict=dict(fontsize=12))
        self.fig.canvas.set_window_title(f'Time_per_parameter_{security_method}')
        self.ax.margins( y= 0.12)
        self.ax.yaxis.set_minor_locator(tck.AutoMinorLocator())

       # self.ax.set(adjustable="datalim")
        #ratio = 0.5
        #xleft, xright = self.ax.get_xlim()
        #ybottom, ytop = self.ax.get_ylim()
        # the abs method is used to make sure that all numbers are positive
        # because x and y axis of an axes maybe inversed.
        #self.ax.set_aspect(abs((xright-xleft)/(ybottom-ytop))*ratio)
        
        set_xmargin(self.ax, left=0.05, right=0.11)
        #set_size(6.4,12.4, self.ax)
        plt.gcf().set_size_inches(5, 7)

        self.fig.tight_layout()
        
        self.fig.savefig(f'Time_per_parameter_{security_method}.pdf', bbox_inches='tight', dpi=1000)
        self.fig.savefig(f'Time_per_parameter_{security_method}.svg', bbox_inches='tight', dpi=1000)
        self.fig.show()
    def anim_func(self, num_bytes, *fargs):
        self.update_fig(num_bytes)
        return flatten(self.bars)
    
    def animate(self, frames=range(100, 6001, 100), save=False, *save_args, **save_kwargs):
        self.animation = animation.FuncAnimation(self.fig, self.anim_func, blit=True, interval=0,frames=frames ,repeat=False)
        if save:
            self.animation.save('Charge_per_Security.mp4', *save_args, **save_kwargs)
        
        
def flatten(d, ret=None):
    if ret is None:
        ret = []
    for k, v in sorted(d.items()):
        if isinstance(v, dict):
            flatten(v, ret)
        else:
            ret.extend(v.get_children())
    return ret

In [95]:
#Security Time Pass same parameters
security_plotter = SecurityPlotter(checkpoint_energy, show=1, use_model=False)

255
255
0 : 159.85389973333497
0 : 172.32475999999508
0 : 88.90289440000032
0 : 8.711648000002015
AES-256_mbedTLS_library_O1_4Mhz: 429.7932021333324
255
255
1 : 53.19769173333455
1 : 57.356235733333705
1 : 58.27617493333648
1 : 2.9041877333320087
AES-256_mbedTLS_library_O1_12Mhz: 171.73429013333674
255
255
2 : 159.7901151999963
2 : 172.25678293333857
2 : 88.56957279999733
2 : 8.778567466670495
AES-256_mbedTLS_library_O3_4Mhz: 429.3950384000027
255
255
3 : 51.28151519999946
3 : 54.714070399999315
3 : 55.90276426666918
3 : 2.7061749333334717
AES-256_mbedTLS_library_O3_12Mhz: 164.60452480000143
